In [1]:
!pip install "sap-llm-commons[all]"

  Obtaining dependency information for langchain[openai]<=0.0.335,>=0.0.331 from https://files.pythonhosted.org/packages/15/f9/e79403efb880425babaa91f8ab19e1b6218dc694551fa6e7f40197b5a72a/langchain-0.0.335-py3-none-any.whl.metadata
  Using cached langchain-0.0.335-py3-none-any.whl.metadata (16 kB)
Using cached langchain-0.0.335-py3-none-any.whl (2.0 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.330
    Uninstalling langchain-0.0.330:
      Successfully uninstalled langchain-0.0.330



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install ai-api-client-sdk


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# proxy configuration
from ipywidgets import widgets
import json
import os
import llm_commons.proxy.base

llm_commons.proxy.base.proxy_version = 'aicore'

In [4]:
from llm_commons.proxy.openai import Completion
from llm_commons.proxy.identity import AICoreProxyClient
from llm_commons.langchain.proxy import ChatOpenAI
from llm_commons.btp_llm.identity import BTPProxyClient
from llm_commons.langchain.proxy import init_llm, init_embedding_model

In [5]:
resource_group = widgets.Text(
    value='8bc16ea0-ef7d-4c7b-9c1e-14da9fcb7d35', # resource group
    placeholder='Resource group of deployments',
    description='',
    disabled=False
)

resource_group

Text(value='8bc16ea0-ef7d-4c7b-9c1e-14da9fcb7d35', placeholder='Resource group of deployments')

In [6]:
with open('config.json') as f:
    sk = json.load(f)

In [7]:
os.environ['AICORE_LLM_AUTH_URL'] = sk['url']+"/oauth/token"
os.environ['AICORE_LLM_CLIENT_ID'] = sk['clientid']
os.environ['AICORE_LLM_CLIENT_SECRET'] = sk['clientsecret']
os.environ['AICORE_LLM_API_BASE'] = sk["serviceurls"]["AI_API_URL"]+ "/v2"
os.environ['AICORE_LLM_RESOURCE_GROUP'] = resource_group.value
os.environ['LLM_COMMONS_PROXY'] = 'aicore'

llm_commons.proxy.resource_group = os.environ['AICORE_LLM_RESOURCE_GROUP']
llm_commons.proxy.api_base = os.environ['AICORE_LLM_API_BASE']
llm_commons.proxy.auth_url = os.environ['AICORE_LLM_AUTH_URL']
llm_commons.proxy.client_id = os.environ['AICORE_LLM_CLIENT_ID']
llm_commons.proxy.client_secret = os.environ['AICORE_LLM_CLIENT_SECRET']

In [8]:
aic_proxy_client = AICoreProxyClient()
btp_proxy_client = BTPProxyClient()

Deploy customized fine-tuned models

In [9]:
AICoreProxyClient.add_foundation_model_scenario(
    scenario_id='fine-tuned-llm',
    config_names='fine-tuned-*',
    prediction_url_suffix='/v1/completions'
)

In [10]:
aic_proxy_client.get_deployments()

[Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d67e66aa880020a1', config_id='c40b53e2-5fbb-4e04-98cc-b240cdec62f9', config_name='text-embedding-ada-002-config', deployment_id='d67e66aa880020a1', model_name='text-embedding-ada-002', created_at=datetime.datetime(2024, 3, 15, 7, 40, 15, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'azure-openai', 'model_version': '2'}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d854d892c8f61bcc', config_id='626bd56e-4b15-4a39-9fbe-11fb27fb04d4', config_name='gpt-4-32k-config', deployment_id='d854d892c8f61bcc', model_name='gpt-4-32k', created_at=datetime.datetime(2024, 3, 15, 7, 40, 15, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'azure-openai', 'model_version': '0613'}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ond

In [11]:
text_davinci_003 = Completion(deployment_id='text-davinci-003')

In [14]:
# text_davinci_003.create(model_name='our-awesome-model/v1', prompt="San Francisco is a")

Initialize harmonized model

In [12]:
llm = init_llm('gpt-35-turbo', proxy_client = aic_proxy_client,temperature=0., max_tokens=256, deployment_id='d4d0fce7c6bdf494', api_base=llm_commons.proxy.api_base)

In [12]:
llm.predict("What is your name?")

"I am an AI language model developed by OpenAI, and I don't have a personal name. You can simply refer to me as OpenAI Assistant. How can I assist you today?"

In [13]:
embedding = init_embedding_model('text-embedding-ada-002-v2', proxy_client=aic_proxy_client, deployment_id='d026d09d4fbbad63', api_base=llm_commons.proxy.api_base)

In [14]:
query = "What is your name?"
query_embedding = embedding.embed_query(query)

In [15]:
documents = ["This is a sample document"]
document_embedding = embedding.embed_documents(documents)

AI Core proxy support

In [16]:
from llm_commons.proxy.base import set_proxy_version
set_proxy_version('aicore') # for an AI Core proxy
from llm_commons.langchain.proxy import ChatOpenAI
chat = ChatOpenAI(proxy_model_name='gpt-4')
chat.predict('What is your name?')

"As an artificial intelligence, I don't have a personal name. I'm simply known as OpenAI."

Proxy client - use proxies in parallel

In [18]:
from llm_commons.langchain.proxy import ChatOpenAI
from langchain.schema.messages import HumanMessage
from llm_commons.proxy.openai import ChatCompletion

In [20]:
messages=[{"role": "user", "content": "Say this is a test"}]

print(ChatCompletion.create(proxy_client=aic_proxy_client, model_name='gpt-4', messages=messages, api_base=os.environ['AICORE_LLM_API_BASE']))

{
  "choices": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      },
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "This is a test.",
        "role": "assistant"
      }
    }
  ],
  "created": 1706510098,
  "id": "chatcmpl-8mFWMxBM0bspGGWshDz3KYobGUIT7",
  "model": "gpt-4",
  "object": "chat.completion",
  "prompt_filter_results": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "jailbreak": {
          "detected": false,
          "filtered": false
        },
        "self_harm": {
          "filtered": false

In [21]:
messages = [
    HumanMessage(content="Say this is a test"),
]
# btp_llm = ChatOpenAI(proxy_client=btp_proxy_client, deployment_id='gpt-4')
# print(btp_llm.invoke(messages))
aic_llm = ChatOpenAI(proxy_client=aic_proxy_client, proxy_model_name='gpt-4')
print(aic_llm.invoke(messages))

content='This is a test.'


Using llm-commons with fine-tuned models

In [23]:
# use it with langchain

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from llm_commons.langchain.proxy import OpenAI

llm = ChatOpenAI(proxy_model_name='gpt-4')
template = "Question: {question}\nAnswer: Let's think step by step."""

In [1]:
prompt = PromptTemplate(template=template, input_variables=['question'])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
llm_chain.run("What is the world's best B2B company?")

NameError: name 'PromptTemplate' is not defined

In [25]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from llm_commons.langchain.proxy import OpenAI

llm = ChatOpenAI(proxy_model_name='gpt-4')
template = "Question: {question}\nAnswer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=['question'])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
llm_chain.run("What is 1+1 ?")




> Entering new LLMChain chain...
Prompt after formatting:
Question: What is 1+1 ?
Answer: Let's think step by step.

> Finished chain.


'First, we have the number 1. Then, we add another 1 to it. When we combine these, we get the number 2. So, 1 + 1 = 2.'

In [26]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [27]:
style = """American English \
in a calm and respectful tone
"""

In [28]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""
print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```


In [29]:
llm_chain.run(prompt)



> Entering new LLMChain chain...
Prompt after formatting:
Question: Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```

Answer: Let's think step by step.

> Finished chain.


"I am quite upset that my blender lid popped off and splattered my kitchen walls with smoothie. To add to the frustration, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend."

In [30]:
from langchain.chat_models import ChatOpenAI

In [34]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""


In [35]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [36]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [37]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [38]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [39]:
customer_style = """American English \
in a calm and respectful tone
"""

In [40]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [41]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>


In [42]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [43]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [44]:
print(customer_response.content)

I'm quite upset that the lid to my blender came off and splattered my kitchen walls with smoothie. To add to the frustration, the warranty does not cover the cost of cleaning up my kitchen. I really need your assistance at this moment.


In [45]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [49]:

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [50]:
conversation.predict(input="Hi, my name is Santhosini")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Santhosini
AI:

> Finished chain.


"Hello, Santhosini! It's a pleasure to meet you. How can I assist you today?"

In [51]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Santhosini
AI: Hello, Santhosini! It's a pleasure to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The sum of 1+1 is 2.'

In [52]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Santhosini
AI: Hello, Santhosini! It's a pleasure to meet you. How can I assist you today?
Human: What is 1+1?
AI: The sum of 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Santhosini.'

In [53]:
print(memory.buffer)

Human: Hi, my name is Santhosini
AI: Hello, Santhosini! It's a pleasure to meet you. How can I assist you today?
Human: What is 1+1?
AI: The sum of 1+1 is 2.
Human: What is my name?
AI: Your name is Santhosini.
